In [33]:
#Dependencies
import pandas as pd
import requests
import json
import datetime
from config import api_key


In [34]:
states_abbrev_file_path = "Resources/states_abbrev.csv"

states_abbrev_df = pd.read_csv(states_abbrev_file_path, encoding='ISO-8859-1')
states_abbrev_df

,Airport,City,Abbrev,State,IATA
0,Aberdeen Regional Airport,Aberdeen,SD,South Dakota,ABR
1,Abilene Regional Airport,Abilene,TX,Texas,ABI
2,Abraham Lincoln Capital Airport,Springfield,IL,Illinois,SPI
3,Akron-Canton Regional Airport,Akron / Canton,OH,Ohio,CAK
4,Albany International Airport,Albany,NY,New York,ALB
...,...,...,...,...,...
367,Yampa Valley Airport,Hayden,CO,Colorado,HDN
368,Yeager Airport,Charleston,WV,West Virginia,CRW
369,Yellowstone Regional Airport,Cody,WY,Wyoming,COD
370,Youngstown-Warren Regional Airport,Youngstown / Warren,OH,Ohio,YNG


In [35]:
top_25_airports = [
    'ATL',
    'LAX',
    'ORD',
    'DFW',
    'DEN',
    'JFK',
    'SFO',
    'SEA',
    'LAS',
    'MCO',
    'EWR',
    'CLT',
    'PHX',
    'IAH',
    'MIA',
    'BOS',
    'MSP',
    'FLL',
    'DTW',
    'PHL',
    'LGA',
    'BWI',
    'SLC',
    'SAN',
    'IAD']


In [36]:
states_abbrev_df = states_abbrev_df.rename(columns = {'Abbrev': 'state_abbrev'})
states_abbrev_df.head()

,Airport,City,state_abbrev,State,IATA
0,Aberdeen Regional Airport,Aberdeen,SD,South Dakota,ABR
1,Abilene Regional Airport,Abilene,TX,Texas,ABI
2,Abraham Lincoln Capital Airport,Springfield,IL,Illinois,SPI
3,Akron-Canton Regional Airport,Akron / Canton,OH,Ohio,CAK
4,Albany International Airport,Albany,NY,New York,ALB


In [37]:
# API Call for covid data 
covid_base_url = "https://api.covidtracking.com/v1/states/daily.json"
response = requests.get(covid_base_url).json()

#print(json.dumps(response, indent=4))

# pull the data from a source df to a target df by creating a new df

covid_df = pd.DataFrame(response, columns=['date', 'state', 'positive', 'death', 'hospitalized', 'totalTestResults'])

#covid_df.fillna(0) replacing the NAN values with zero
covid_df['hospitalized'] =covid_df['hospitalized'].fillna(0)

covid_df.head()


,date,state,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,0.0,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,0.0,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [38]:
covid_df = covid_df.rename(columns = {'state': 'state_abbrev'})

covid_df.head()


,date,state_abbrev,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,0.0,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,0.0,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [39]:
covid_df['Date'] = ''
covid_df.head()

        

,date,state_abbrev,positive,death,hospitalized,totalTestResults,Date
0,20201021,AK,12644.0,68.0,0.0,547140.0,
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0,
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0,
3,20201021,AS,0.0,0.0,0.0,1616.0,
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0,


In [40]:

for x, y in covid_df.iterrows():
    
    org_date = str(y.get(key='date'))
    
    year_s = org_date[0:4]
    month_s = org_date[4:6]
    day_s = org_date[6:8]
    formatted_date = f"{year_s}-{month_s}-{day_s}"
    
    covid_df.at[x, 'Date'] = str(formatted_date)
  
    
        
covid_df = covid_df.drop('date', axis=1)

covid_df.head(20)


,state_abbrev,positive,death,hospitalized,totalTestResults,Date
0,AK,12644.0,68.0,0.0,547140.0,2020-10-21
1,AL,174528.0,2805.0,19330.0,1273523.0,2020-10-21
2,AR,101596.0,1751.0,6526.0,1242752.0,2020-10-21
3,AS,0.0,0.0,0.0,1616.0,2020-10-21
4,AZ,233912.0,5854.0,20820.0,1657389.0,2020-10-21
5,CA,877784.0,17027.0,0.0,17293139.0,2020-10-21
6,CO,87582.0,2060.0,8332.0,1724038.0,2020-10-21
7,CT,64871.0,4567.0,12043.0,2070126.0,2020-10-21
8,DC,16498.0,642.0,0.0,476443.0,2020-10-21
9,DE,23375.0,670.0,0.0,331480.0,2020-10-21


In [41]:


no_of_days = 5

curr_date = pd.to_datetime('today').date()
curr_date =  pd.to_datetime(curr_date)


#START DATE 
start_date = str(curr_date)
start_date = start_date.split(' ')[0]


#END DATE
end_date = datetime.datetime.today() - datetime.timedelta(days=no_of_days)
end_date = str(end_date)
end_date = end_date.split(' ')[0]

print(start_date)
print(end_date)


2020-10-22
2020-10-17


In [42]:
#Generate a list that will hold all the flight dates we would like to query

curr_date =datetime.datetime.today() - datetime.timedelta(days=no_of_days)
query_dates = []

for i in range(1,no_of_days + 1):
    

    Next_date = curr_date + datetime.timedelta(days=1)

    str_date =  pd.to_datetime(Next_date)
    str_date = str(str_date)
    str_date = str_date.split(' ')[0]
    
    curr_date = Next_date
    query_dates.append(str_date)
    
query_dates
   

['2020-10-18', '2020-10-19', '2020-10-20', '2020-10-21', '2020-10-22']

In [43]:
# API Call for flights data 

url = f"http://api.aviationstack.com/v1/flights?access_key={api_key}&"
#query_date = ['2020-10-01','2020-10-02','2020-10-03' ]
query_iata = ['ATL','LAX','ORD']
#query_iata = top_25_airports
value_list = []

for q_date in query_dates:
    for q_iata in query_iata:
        flights_base_url = f"{url}flight_date={q_date}&arr_iata={q_iata}&flight_status=landed&limit=1"
        flights_response = requests.get(flights_base_url).json()
        landed_flights = flights_response['pagination']['total']
        arr_IATA = flights_response['data'][0]['arrival']['iata']
    
       
        value_list.append({'Date': q_date, 'IATA':arr_IATA, 'Total Landed Flights': landed_flights})
        
flights_df = pd.DataFrame(value_list)   
flights_df


,Date,IATA,Total Landed Flights
0,2020-10-18,ATL,2454
1,2020-10-18,LAX,1184
2,2020-10-18,ORD,1996
3,2020-10-19,ATL,2152
4,2020-10-19,LAX,1106
5,2020-10-19,ORD,1861
6,2020-10-20,ATL,1998
7,2020-10-20,LAX,996
8,2020-10-20,ORD,1507
9,2020-10-21,ATL,1853


In [44]:
#Merge states data with flights data

merged_df = states_abbrev_df.merge(flights_df, on='IATA')
merged_df.head(50)

,Airport,City,state_abbrev,State,IATA,Date,Total Landed Flights
0,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-18,1996
1,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-19,1861
2,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-20,1507
3,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-21,1149
4,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-22,420
5,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-18,2454
6,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-19,2152
7,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-20,1998
8,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-21,1853
9,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-22,635


In [45]:
#Merge Flight data with COVID data

merged_df = merged_df.merge(covid_df, how='inner', left_on=["Date", "state_abbrev"], right_on=["Date","state_abbrev"])
merged_df

,Airport,City,state_abbrev,State,IATA,Date,Total Landed Flights,positive,death,hospitalized,totalTestResults
0,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-18,1996,347635.0,9474.0,0.0,6779140.0
1,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-19,1861,350748.0,9496.0,0.0,6827824.0
2,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-20,1507,354462.0,9537.0,0.0,6886901.0
3,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-21,1149,358804.0,9605.0,0.0,6953692.0
4,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-18,2454,340558.0,7638.0,30376.0,3298458.0
5,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-19,2152,341310.0,7657.0,30388.0,3314279.0
6,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-20,1998,342438.0,7674.0,30541.0,3327836.0
7,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-21,1853,342438.0,7674.0,30541.0,3327836.0
8,Los Angeles International Airport,Los Angeles,CA,California,LAX,2020-10-18,1184,867317.0,16943.0,0.0,16892062.0
9,Los Angeles International Airport,Los Angeles,CA,California,LAX,2020-10-19,1106,870791.0,16970.0,0.0,17042408.0


In [70]:
#adding column of total people infected and converting to int
merged_df["Total Infected"] = merged_df["death"] + merged_df["positive"] + merged_df["hospitalized"]
merged_df["Total Infected"] = merged_df["Total Infected"].astype(int)
merged_df

,Airport,City,state_abbrev,State,IATA,Date,Total Landed Flights,positive,death,hospitalized,totalTestResults,Month,Total Infected
0,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-18,1996,347635.0,9474.0,0.0,6779140.0,10,357109
1,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-19,1861,350748.0,9496.0,0.0,6827824.0,10,360244
2,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-20,1507,354462.0,9537.0,0.0,6886901.0,10,363999
3,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-21,1149,358804.0,9605.0,0.0,6953692.0,10,368409
4,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-18,2454,340558.0,7638.0,30376.0,3298458.0,10,378572
5,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-19,2152,341310.0,7657.0,30388.0,3314279.0,10,379355
6,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-20,1998,342438.0,7674.0,30541.0,3327836.0,10,380653
7,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-21,1853,342438.0,7674.0,30541.0,3327836.0,10,380653
8,Los Angeles International Airport,Los Angeles,CA,California,LAX,2020-10-18,1184,867317.0,16943.0,0.0,16892062.0,10,884260
9,Los Angeles International Airport,Los Angeles,CA,California,LAX,2020-10-19,1106,870791.0,16970.0,0.0,17042408.0,10,887761


In [71]:
#adding a new colomn to display the month
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df["Month"] = merged_df["Date"].dt.month
merged_df


,Airport,City,state_abbrev,State,IATA,Date,Total Landed Flights,positive,death,hospitalized,totalTestResults,Month,Total Infected
0,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-18,1996,347635.0,9474.0,0.0,6779140.0,10,357109
1,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-19,1861,350748.0,9496.0,0.0,6827824.0,10,360244
2,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-20,1507,354462.0,9537.0,0.0,6886901.0,10,363999
3,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-21,1149,358804.0,9605.0,0.0,6953692.0,10,368409
4,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-18,2454,340558.0,7638.0,30376.0,3298458.0,10,378572
5,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-19,2152,341310.0,7657.0,30388.0,3314279.0,10,379355
6,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-20,1998,342438.0,7674.0,30541.0,3327836.0,10,380653
7,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-21,1853,342438.0,7674.0,30541.0,3327836.0,10,380653
8,Los Angeles International Airport,Los Angeles,CA,California,LAX,2020-10-18,1184,867317.0,16943.0,0.0,16892062.0,10,884260
9,Los Angeles International Airport,Los Angeles,CA,California,LAX,2020-10-19,1106,870791.0,16970.0,0.0,17042408.0,10,887761


In [72]:
project_df = pd.DataFrame({"Date": merged_df['Date'], "Month": merged_df["Month"], "State": merged_df["State"],
                          "Number of Flights": merged_df["Total Landed Flights"], "People Infected": merged_df["Total Infected"]})
project_df

,Date,Month,State,Number of Flights,People Infected
0,2020-10-18,10,Illinois,1996,357109
1,2020-10-19,10,Illinois,1861,360244
2,2020-10-20,10,Illinois,1507,363999
3,2020-10-21,10,Illinois,1149,368409
4,2020-10-18,10,Georgia,2454,378572
5,2020-10-19,10,Georgia,2152,379355
6,2020-10-20,10,Georgia,1998,380653
7,2020-10-21,10,Georgia,1853,380653
8,2020-10-18,10,California,1184,884260
9,2020-10-19,10,California,1106,887761


In [ ]:
#calculating total according to the month
total_infection = project_df.groupby(["Month"]).sum()["People Infected"]
landing_flights = project_df.groupby(["Month"]).sum()["Number of Flights"]
month_data_df = pd.DataFrame({"Total Infection": total_infection, "Landed Flights": landing_flights})
month_data_df
